In [149]:
import re
import jieba
import jieba.analyse
import json
import random
import operator
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [150]:
from elasticsearch import Elasticsearch
import elasticsearch.helpers
from datetime import datetime

In [151]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [152]:
# Find top10
res = es.search(index='facebook', doc_type='user', body= {'query': {'match': {'gender': 'female'}}})
print("%d documents found" % res['hits']['total'])

58 documents found


In [153]:
# Take all result
users = list(elasticsearch.helpers.scan(es, index='facebook', doc_type='user'))
total_users = [user['_id'] for user in users]
total_user_with_gender = [(user['_id'], user['_source'].get('gender')) for user in users]
print(len(total_user_with_gender))

143


In [154]:
no_message = ['1618047998214350', '1624029027647185', '1454298984658260']
foreigners = ['1695358370488750', '10213401737335928', '1532260126834371', '10154597524691479', '10155404471078548', '10158882655745234']

In [155]:
total_users = [user for user in total_users if user not in (no_message + foreigners)]
print(len(total_users))

134


In [156]:
user_dict = {'male': [], 'female': []}
for user in total_user_with_gender:
    if user[0] not in (no_message + foreigners):
        if user[1] == 'male':
            user_dict['male'].append(user[0])
        else:
            user_dict['female'].append(user[0])

In [157]:
print('male: ' + str(len(user_dict['male'])) + '\nfemale: ' + str(len(user_dict['female'])))

male: 80
female: 54


In [158]:
# Filtering urls
def filtering_url(messages):
    filtered_messages = [re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', message.replace('\n', '').replace('\r', '')) for message in messages]
    return filtered_messages

In [159]:
# Mapping user posts
def get_posts(users):
    posts_dic = {}
    for user in users:
        # print(user)
        posts = list(elasticsearch.helpers.scan(es, query={'query': {'match': {'user_id': user}}}, index='facebook', doc_type='post'))
        messages = [post['_source'].get('message') for post in posts if post['_source'].get('message') != '']
        #print(message)
        filtered_messages = filtering_url(messages)
        posts_dic[user] = filtered_messages
    return posts_dic

In [160]:
total_posts = get_posts(total_users)
male_posts_dict = get_posts(user_dict['male'])
female_posts_dict = get_posts(user_dict['female'])

# Using jieba as Chinese tokenizer, and calculating TFIDF

In [12]:
# Using jieba to tokenize user posts
def tokenize_posts(user_posts):
    seg_posts = {}
    for key, messages in user_posts.items():
        seg_posts[key] = [' '.join(jieba.cut(message, cut_all = False)) for message in messages]
    return seg_posts

In [13]:
male_seg_posts = tokenize_posts(male_posts_dict)
female_seg_posts = tokenize_posts(female_posts_dict)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.685 seconds.
Prefix dict has been built succesfully.


In [14]:
# Display tfidf score
def display_scores(vectorizer, tfidf_result):
    # http://stackoverflow.com/questions/16078015/
    scores = zip(vectorizer.get_feature_names(),
                 np.asarray(tfidf_result.sum(axis=0)).ravel())
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    index_scores = []
    for index, item in enumerate(sorted_scores):
        index_scores.append((index, item[1]))
    return sorted_scores

In [15]:
def calculate_tfidf(seg_posts):
    corpus = [segment for key, segments in seg_posts.items() for segment in segments]
    tfidf_vectorizer = TfidfVectorizer()
    tfidf = tfidf_vectorizer.fit_transform(corpus)
    tfidf_features = tfidf_vectorizer.get_feature_names()
    
    sorted_scores = display_scores(tfidf_vectorizer, tfidf)
    return sorted_scores

In [16]:
male_tfidf_scores = calculate_tfidf(male_seg_posts)
female_tfidf_scores = calculate_tfidf(female_seg_posts)

In [17]:
male_bag_of_words = [word[0] for word in male_tfidf_scores[:100]]
female_bag_of_words = [word[0] for word in female_tfidf_scores[:100]]

# Graph-based word categorization

In [88]:
%%time
word_index = {}
index = 1

#Building word index dictionary
for key, messages in total_posts.items():
    for message in messages:
        for word in list(message):
            if word not in word_index:
                word_index[word] = index
                index += 1

CPU times: user 484 ms, sys: 0 ns, total: 484 ms
Wall time: 500 ms


In [120]:
def building_weighted_graph(word_graph):
    weighted_word_graph = {}
    # Finding the maximun value of word frequency
    max_freq = max(word_graph.items(), key=operator.itemgetter(1))[1]

    for key, value in word_graph.items():
        weight = value / max_freq
        weighted_word_graph[key] = weight 
    return weighted_word_graph

In [121]:
def building_word_graph(user_posts):
    word_graph = {}
    for key, messages in user_posts.items():
        for message in messages:
            words = list(message)
            
            for i in range(len(words)-1):
                if (words[i], words[i+1]) not in word_graph:
                    word_graph[(words[i], words[i+1])] = 1
                else:
                    word_graph[(words[i], words[i+1])] += 1
    weighted_word_graph = building_weighted_graph(word_graph)
    return weighted_word_graph

In [122]:
male_weighted_graph = building_word_graph(male_posts_dict)
female_weighted_graph = building_word_graph(female_posts_dict)

In [123]:
import networkx as nx
import plotly.plotly as py
from plotly.graph_objs import *

In [124]:
G = nx.random_geometric_graph(200, 1)
# position is stored as node attribute data for random_geometric_graph
pos = nx.get_node_attributes(G, 'pos')

In [125]:
edge_trace = Scatter(
    x=[], 
    y=[], 
    line=Line(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

In [126]:
for key, messages in male_posts_dict.items():
    for message in messages:
        words = list(message)
        if '.' in words:
            print(message)
            #print(words)
            break

2016 回顧感覺浪費了很多時間唯一值得的是不知道練了幾隻舞更確定自己的興趣唯一在跳舞才能找到自己的自信即使被嘲笑 也無所謂 畢竟是自己喜歡的事情管他別人的眼光 -希望 2017 能更順利期望明年順利畢業與找到工作然後身體健康 老掉牙卻很實在的願望-然後大家新年快樂😘😘😘-最後附上今年練舞歌單 :BLACKPINK - PLAYING WITH FIREI.O.I - VERY *3TWICE - TTOMG - WINDY DAYWJSN - SECRETEXID - L.I.EGFRIEND - NAVILLERASEVENTEEN - ADORE UTWICE - CHEER UPI.O.I - CRUSHGFRIEND - ROUGHTWICE - Like OOH-AHH
啦啦啦啦啦開始懷念布達佩斯的店員了...
你以為你多益聽力對多國口音很強嗎? 聽完這個你就哭了...
宇智波班到底是....
HBL高級中等學校籃球甲級聯賽松山 v 高苑 主播阿萌 ft. 賽評狗熠 熱身賽前講評
都快開學了 竟然備取上台大了...
［培養忠誠屬下：不要讓他做白工］跟著公司和非營利組織做量化研究已經進入到一定的階段，本來只是想利用上面的人和和某週刊對企業有建立聯繫管道，請特定樣本的企業填一份問卷。不過上面的人和週刊執行長很麻吉，把我們的想法告訴週刊以後，不僅要我們提案，還要我們為了配合週刊主筆部門與行銷部門對我們的研究在分別對不同區塊有興趣，無論他們最後哪部門勝出都要我們盡力說服，修改研究方向。總而言之提了一些天馬行空的提案，你也知道的，出錢的人的日常。而且還自己想像那個周刊只有做跟產品/行銷有關的封面故事，殊不知我前天才剛看完他們歐洲難民的專題。                                                      . . .「我們的小孩目前已經成形，也花時間做了，大概就是這樣，化個妝還可以，不要想要整形，很難，我們要有很清楚的研究背景，也不要去期待可以用他們的什麼資料，不然他們自己也有研究部門，為什麼要跟我們合作。」另外一個我比較欣賞的高層說道。                                                     . . .有一個很有名的實驗，管理學上也常常提到，這邊用親子天下的專欄來講這個故事：

# Topic Modeling

In [168]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [169]:
corpus = []
for key, messages in total_posts.items():
    corpus += [' '.join(jieba.cut(message, cut_all = False)) for message in messages]

In [170]:
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(corpus)

In [171]:
tf_vectorizer = CountVectorizer()
tf = tf_vectorizer.fit_transform(corpus)

In [172]:
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [173]:
lda = LatentDirichletAllocation(n_topics=10, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [174]:
lda.fit(tf)

/usr/local/lib/python3.5/dist-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning:

n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21



LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=10, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [175]:
tf_feature_names = tf_vectorizer.get_feature_names()

In [176]:
print_top_words(lda, tf_feature_names, 20)

Topic #0:
ahq 12 鎖鎖鎖 正在 est le 加入 un 快下班 lab 期中考 je en 腦袋 愛的 pas 相機 der oaq 一小
Topic #1:
參加 活動 xd 小孩 xddd 圖靈機 媽媽 相信 晚餐 神魔 根本 路上 資料 到底 晚安 好多 快來 肚子 爸爸 舒服
Topic #2:
真的 可以 自己 一個 我們 大家 今天 什麼 還是 知道 就是 這樣 不是 一直 因為 覺得 時候 這個 謝謝 希望
Topic #3:
哈哈 感覺 工作 世界 甚麼 公司 一點 支持 手機 一起 機會 天氣 下午 我要 自己 出來 突然 越來 決定 哪裡
Topic #4:
朋友 他們 只有 生活 哈哈哈 台灣 每次 昨天 回家 果然 成功 一切 不能 幸福 特別 總是 睡覺 感覺 政府 我們
Topic #5:
開心 真是 整個 今天 不要 影片 這裡 不好 50 而已 出門 早餐 一起 100 程式 看著 當然 分享 習慣 qaq
Topic #6:
原來 期待 開始 日本 就要 看來 la 如此 能夠 了解 day 慢慢 lw 一次 那邊 幫助 居然 存在 現實 文化
Topic #7:
uw rw the b2 l2 r2 d2 u2 f2 uw2 fw2 rw2 to you and of in my is time
Topic #8:
看到 一下 有人 可能 第一次 想要 準備 晚上 回來 剛剛 怎麼 11 10 明天 記得 30 竟然 20 終於 怎樣
Topic #9:
fw 明天 第一 好吃 學校 超級 啊啊啊 電影 台南 qq fb 報告 tpa 不知 今天 常常 音樂 好看 第二 ya

